In [1]:
# # Install required packages and start virtual display
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y
!pip install selenium
!pip install webdriver-manager
!apt-get install -y xvfb
!pip install pyvirtualdisplay
!pip install beautifulsoup4
!pip install pandas

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [2]:
# Import necessary modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyvirtualdisplay import Display
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive


display = Display(visible=0, size=(1024, 768))
display.start()

# Set up the Selenium Chrome driver with Chrome binary path
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the target URL
url = "https://outages.ncelectriccooperatives.com/outages/maps"
driver.get(url)

# Wait for the page to load
time.sleep(10)  # Adjust as needed depending on your internet speed

# Extract everything from the div with class "jsCountyDetailsWrapperByAlpha"
try:
    # Wait until the div is present in the DOM
    county_details_wrapper = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'jsCountyDetailsWrapperByAlpha'))
    )

    # Get the HTML content of the div
    county_details_html = county_details_wrapper.get_attribute('innerHTML')

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(county_details_html, 'html.parser')

    # Prepare lists to hold data
    counties = []
    members_affected = []
    utilities = []

    # Extract information from the HTML content
    outage_reports = soup.find_all('dl', class_='outage_report')

    for report in outage_reports:
        # Extract county name
        county_name = report.find('dt').text.strip()

        # Extract number of members affected
        affected_info = report.find('p').text.strip().split('member')[0]

        # Extract the utility information (if available)
        utility_info_list = report.find_all('li')
        utility_info = ", ".join([utility.text.strip().split(':')[0] for utility in utility_info_list])

        # Append data to lists
        counties.append(county_name)
        members_affected.append(affected_info)
        utilities.append(utility_info)

    # Create a DataFrame from the extracted data
    data = {
        'Name': counties,
        'Members_Affected': members_affected,
        'Utility':utilities,
        'County': [c.split('(')[0] for c in counties]
    }
    df = pd.DataFrame(data)

except Exception as e:
    print("An error occurred:", e)

# Clean up
driver.quit()
display.stop()


In [3]:
df

,Name,Members_Affected,Utility,County
0,Brunswick County (NC),25,Brunswick Electric Membership Corporation,Brunswick County
1,Buncombe County (NC),7,French Broad Electric Membership Corporation,Buncombe County
2,Columbus County (NC),11,Brunswick Electric Membership Corporation,Columbus County
3,Gates County (NC),1,Roanoke Electric Cooperative,Gates County
4,Harnett County (NC),3,Central Electric Membership Corporation,Harnett County
5,Madison County (NC),166,French Broad Electric Membership Corporation,Madison County
6,McDowell County (NC),1,French Broad Electric Membership Corporation,McDowell County
7,Mitchell County (NC),62,French Broad Electric Membership Corporation,Mitchell County
8,Moore County (NC),1,Pee Dee Electric Membership Corporation,Moore County
9,Scotland County (NC),2,Pee Dee Electric Membership Corporation,Scotland County


In [4]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to save the file in Google Drive
output_path = '/content/drive/My Drive/outage_details_by_county.csv'

# Save the DataFrame to CSV
df.to_csv(output_path, index=False)

print(f"Data exported successfully to Google Drive: {output_path}")

Mounted at /content/drive
Data exported successfully to Google Drive: /content/drive/My Drive/outage_details_by_county.csv
